In [1]:
import sys
sys.path.append('/home/npopkov/dll24')

from models.ddpm.DDPMBase import DDPMBase
from models.ddpm.CAUnet import CAUnet, UNet_conditional

import matplotlib.pyplot as plt
import torch
torch.backends.cudnn.benchmark = True

import numpy as np
import lightning as lt
torch.set_float32_matmul_precision('medium')
import soundfile as sf
from datasets import load_dataset
dataset = load_dataset('DBD-research-group/BirdSet','HSN')
file_path = dataset['test_5s'][356]['filepath']
sample, samplerate = sf.read(file_path)
sample = torch.tensor(sample,device='cuda',dtype=torch.float32)

In [2]:
batch_size = 1
n_epochs = 40
lr = 3e-4
n_steps, betaMin, betaMax = 200, 10**-6, 0.002

network = UNet_conditional(n_steps=n_steps,
                 c_in=1,
                 c_out=1,).to('cuda')

ddpm = DDPMBase(network=network,
                betaMin=betaMin,
                betaMax=betaMax,
                n_steps=n_steps).to('cuda')

In [3]:
from torch.utils.data import DataLoader,Dataset

class fun_dataset(Dataset):
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
    
    def __len__(self):
        return 64
        
    def __getitem__(self, idx):
        return self.data

#dataset = fun_dataset(sample, batch_size)
#train_loader = DataLoader(dataset, batch_size=64, shuffle=True,num_workers=30)

In [4]:
test = ddpm.predictNoise(sample[:25000].unsqueeze(0).unsqueeze(0).to('cuda'),torch.tensor([200]).to('cuda'),None,None)

OutOfMemoryError: CUDA out of memory. Tried to allocate 9.31 GiB. GPU 0 has a total capacty of 23.65 GiB of which 6.09 GiB is free. Including non-PyTorch memory, this process has 17.54 GiB memory in use. Of the allocated memory 16.11 GiB is allocated by PyTorch, and 997.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [ ]:
#trainer = lt.Trainer(max_epochs=n_epochs)
#trainer.fit(ddpm, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params
---------------------------------------------
0 | network | UNet_conditional | 6.1 M 
---------------------------------------------
6.1 M     Trainable params
0         Non-trainable params
6.1 M     Total params
24.303    Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.22 GiB. GPU 0 has a total capacty of 23.65 GiB of which 1.06 GiB is free. Including non-PyTorch memory, this process has 22.57 GiB memory in use. Of the allocated memory 22.11 GiB is allocated by PyTorch, and 12.74 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF